In [ ]:
import numpy as np
import pandas as pd
import math

In [ ]:
train_df = pd.read_csv('mush_train.data', header=None)
test_df = pd.read_csv('mush_test.data', header=None)
mush_train = train_df.to_numpy()
mush_test = test_df.to_numpy()
print(mush_train.shape, mush_test.shape)
X_train = mush_train[:, 1:]
Y_train = mush_train[:, 0]
X_test = mush_test[:, 1:]
Y_test = mush_test[:, 0]
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

(4712, 23) (3412, 23)
(4712, 22) (4712,) (3412, 22) (3412,)


In [ ]:
def H(X):
  entropy = 0

  for x in np.unique(X):
    p = (X == x).sum() / len(X)

    if p > 0:
      entropy -= p * math.log(p)
  
  return entropy

In [ ]:
def condH(Y, X):
  entropy = 0

  for x in np.unique(X):
    subset = Y[X == x]
    s = 0

    for y in np.unique(Y):
      p = (subset == y).sum() / len(subset)

      if p > 0:
        s += p * math.log(p)
    
    entropy += s * (X == x).sum() / len(X)
  
  return -entropy

In [ ]:
def ig(Y, X):
  return H(Y) - condH(Y, X)

In [ ]:
class DecisionTree:
  def __init__(self, X, Y, depth=0, maxdepth=16):
    self.value = None
    self.X = X
    self.Y = Y

    if len(np.unique(self.Y)) == 1:
      self.value = self.Y[0]
    elif depth == maxdepth:
      vals, counts = np.unique(self.Y, return_counts=True)
      self.value = vals[np.argwhere(counts == np.max(counts))].flatten()[0]
    else:
      self.attr = None
      best_ig = None

      for i in range(self.X.shape[1]):
        attr_ig = ig(self.Y, self.X[:, i])

        if self.attr == None or attr_ig > best_ig:
          self.attr = i
          best_ig = attr_ig
      
      self.threshold = self.X[:, self.attr].mean()
      ge_i = np.where(self.X[:, self.attr] >= self.threshold)[0]

      if len(ge_i) > 0:
        self.ge = DecisionTree(self.X[ge_i], self.Y[ge_i], depth=depth + 1, maxdepth=maxdepth)

      lt_i = np.where(self.X[:, self.attr] < self.threshold)[0]
      
      if len(lt_i) > 0:
        self.lt = DecisionTree(self.X[lt_i], self.Y[lt_i], depth=depth + 1, maxdepth=maxdepth)

      if len(lt_i) == 0:
        self.lt = self.ge
      elif len(ge_i) == 0:
        self.ge = self.lt
  
  def predict(self, X):
    if self.value != None:
      return self.value
    
    if X[self.attr] >= self.threshold:
      return self.ge.predict(X)
    
    return self.lt.predict(X)

In [ ]:
X = np.array([[1, 1], [1, 0], [1, 1], [1, 0], [0, 1], [0, 0], [0, 1], [0, 0]])
Y = np.array([1, 1, 1, 1, 1, 0, 0, 0])
tree = DecisionTree(X, Y)
(np.array([tree.predict(x) for x in X]) == Y).mean()

0.875

In [ ]:
def encode(X, Y):
  for i in range(X.shape[1]):
    unique = np.unique(X[:, i]).tolist()

    for j in range(len(X)):
      X[j, i] = unique.index(X[j, i])

  unique = np.unique(Y).tolist()

  for j in range(len(Y)):
    Y[j] = unique.index(Y[j])

encode(X_train, Y_train)
encode(X_test, Y_test)
print(X_train)
print(Y_train)

[[2 0 4 ... 3 5 0]
 [5 3 9 ... 3 5 4]
 [5 3 4 ... 2 3 1]
 ...
 [2 0 4 ... 2 3 1]
 [5 0 9 ... 1 5 1]
 [5 0 3 ... 3 4 0]]
[0 0 1 ... 0 1 0]


In [ ]:
model = DecisionTree(X_train, Y_train, maxdepth=16)

In [ ]:
([model.predict(x) for x in X_test] == Y_test).mean()

1.0